# Snowflake connection parameters

In [11]:
import os
from snowflake.snowpark import Session
import plotly.express as px
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

SNOWFLAKE_ACCOUNT = os.getenv("SNOWFLAKE_ACCOUNT")
SNOWFLAKE_USER = os.getenv("SNOWFLAKE_USER")
SNOWFLAKE_PASSWORD = os.getenv("SNOWFLAKE_PASSWORD")
SNOWFLAKE_DATABASE = os.getenv("SNOWFLAKE_DATABASE")
SNOWFLAKE_WH = os.getenv("SNOWFLAKE_WH")
SNOWFLAKE_ROLE = os.getenv("SNOWFLAKE_ROLE")

GOLD_SCHEMA = os.getenv("GOLD_SCHEMA", "MARTS")

SILVER_SCHEMA = os.getenv("SILVER_SCHEMA", "CORE")

connection_parameters = {
    "account": SNOWFLAKE_ACCOUNT,
    "user": SNOWFLAKE_USER,
    "password": SNOWFLAKE_PASSWORD,
    "database": SNOWFLAKE_DATABASE,
    "warehouse": SNOWFLAKE_WH,
    "role": SNOWFLAKE_ROLE,
    "schema": GOLD_SCHEMA  # default schema set to GOLD (e.g. MARTS)
}

# Create a Snowpark session
session = Session.builder.configs(connection_parameters).create()

# Chart 1: Sales by Day (Line Chart)

In [15]:
query1 = f"""
    SELECT order_date, total_sales
    FROM {GOLD_SCHEMA}.sales_by_day
    ORDER BY order_date
"""
df1 = session.sql(query1).to_pandas()
fig1 = px.line(df1, x="ORDER_DATE", y="TOTAL_SALES", title="Sales by Day", line_shape="spline")
fig1.show()

# Chart 2: Top 10 Products by Sales (Bar Chart)

In [9]:
import plotly.express as px

# Run the query and convert to pandas DataFrame
query2 = f"""
    SELECT product_name, total_sales
    FROM {GOLD_SCHEMA}.sales_by_product
    ORDER BY total_sales DESC
    LIMIT 10
"""
df2 = session.sql(query2).to_pandas()

# Create a horizontal bar chart with improved visuals
fig2 = px.bar(
    df2, 
    x="TOTAL_SALES", 
    y="PRODUCT_NAME", 
    orientation="h",
    title="Top 10 Products by Sales",
    text="TOTAL_SALES", 
    color="TOTAL_SALES", 
    color_continuous_scale=px.colors.sequential.Viridis
)

# Update layout for clarity
fig2.update_layout(
    xaxis_title="Total Sales",
    yaxis_title="Product Name",
    yaxis=dict(autorange="reversed")  # Highest sales on top
)

# Format text labels
fig2.update_traces(
    texttemplate='%{text:.2s}', 
    textposition='auto'
)

fig2.show()


# Chart 3: Top 10 Stores by Order Count (Bar Chart)

In [8]:
query3 = f"""
    SELECT store_name, total_orders
    FROM {GOLD_SCHEMA}.sales_by_store
    ORDER BY total_orders DESC
    LIMIT 10
"""
df3 = session.sql(query3).to_pandas()

# Optionally, if needed ensure column names are uppercase to match below references
df3.columns = [col.upper() for col in df3.columns]

fig3 = px.bar(
    df3,
    x="STORE_NAME",
    y="TOTAL_ORDERS",
    title="Top 10 Stores by Order Count",
    labels={"STORE_NAME": "Store Name", "TOTAL_ORDERS": "Total Orders"},
    text="TOTAL_ORDERS",  # display the value on top of the bars
    template="plotly_white",  # clean white background template
    hover_data={"TOTAL_ORDERS": True, "STORE_NAME": False}
)

# Format the text annotations and adjust the layout
fig3.update_traces(texttemplate='%{text}', textposition='outside')
fig3.update_layout(
    xaxis_tickangle=-45,  # rotate x-axis labels for better readability
    uniformtext_minsize=10,
    uniformtext_mode='hide',
    margin=dict(l=40, r=40, t=60, b=80)
)

fig3.show()


# Chart 4: Top 10 Customers by Lifetime Spend (Scatter Plot)

In [4]:
import plotly.express as px

query4 = f"""
    SELECT customer_id, lifetime_spend
    FROM {GOLD_SCHEMA}.dim_customer_profile
    WHERE lifetime_spend > 0
    ORDER BY lifetime_spend DESC
    LIMIT 10
"""
df4 = session.sql(query4).to_pandas()

# Sort the dataframe so that the smallest value is on top for the horizontal bar chart.
df4_sorted = df4.sort_values(by='LIFETIME_SPEND', ascending=True)

# Create a horizontal bar chart
fig4 = px.bar(
    df4_sorted,
    x="LIFETIME_SPEND",
    y="CUSTOMER_ID",
    orientation="h",
    title="Top 10 Customers by Lifetime Spend",
    labels={
        "LIFETIME_SPEND": "Lifetime Spend ($)",
        "CUSTOMER_ID": "Customer ID"
    }
)

fig4.update_layout(
    xaxis_tickprefix="$",
    yaxis_title="",
    template="plotly_white"
)
fig4.show()


# Chart 5: Order Status Distribution (Pie Chart)

In [7]:
query5 = f"""
    SELECT order_status, COUNT(*) AS count_status
    FROM {SILVER_SCHEMA}.fact_orders
    GROUP BY order_status
"""
df5 = session.sql(query5).to_pandas()

# Create a pie chart (donut style) with improved text info and layout
fig5 = px.pie(
    df5,
    names="ORDER_STATUS",
    values="COUNT_STATUS",
    title="Order Status Distribution",
    color_discrete_sequence=px.colors.qualitative.Plotly
)

# Convert to a donut chart and update text settings
fig5.update_traces(
    hole=0.4,                # Creates a donut chart
    textposition='inside',
    textinfo='percent+label' # Displays both percentage and label on slices
)

# Adjust layout for a cleaner look
fig5.update_layout(
    margin=dict(t=50, b=0, l=0, r=0),
    uniformtext_minsize=16,
    uniformtext_mode='hide'
)

fig5.show()
